# General discovery over the Binance API

In [81]:
import numpy as np
import math
import time
import pandas as pd
from datetime import datetime

In [1]:
from trading.assets.binance import Binance

# Future call

In [48]:
def rounding( number, decimals, mode = "down" ):
    decimals_exp = 10**decimals
    if mode == "down":
        v = math.floor( number*decimals_exp ) / decimals_exp
    elif mode == "up":
        v = math.ceil( number*decimals_exp ) / decimals_exp
    else:
        raise ValueError(f"No mode {mode}")
    
    if v == float(f'%.{decimals}f'%(number)):
        return round(number, decimals)
    else:
        print(v, float(f'%.{decimals}f'%(number)))
        return v

In [153]:
bi = Binance(symbol="")

In [168]:
symbol = "{}USDT".format("APT")
pct = 1.0025
share = .35
leverage = 20

In [169]:
balance = float([ i["balance"] for i in bi.client.futures_account_balance() if i["asset"] == "USDT"][0])
price = bi.client.futures_symbol_ticker(symbol = symbol)["price"]
price = float(price)

price, balance

(3.8368, 12.0121723)

In [170]:
ticker_info = bi.client.get_symbol_info(symbol)

qty_rouding = len(str(float([i["stepSize"] for i in ticker_info["filters"] if i["filterType"] == "LOT_SIZE"][0])).split(".")[-1])
qty_rouding

2

In [171]:
qty_rouding-=1

In [172]:
try:
    qty = balance*leverage*share / price
    qty = round( qty, qty_rouding   )

    bi.client.futures_change_leverage(symbol=symbol, leverage=leverage)

    orderBuy = bi.client.futures_create_order(
        symbol = symbol,
        type = "MARKET",
        # timeInForce ="GTC",
        side = "BUY",
        quantity = qty,
    )
except Exception as e:
    print(e)

In [178]:
df_trades = pd.DataFrame(bi.client.futures_account_trades())
df_trades = df_trades[ df_trades["orderId"] == orderBuy["orderId"] ]
real_price_bought = float(df_trades["price"].iloc[-1])
real_price_bought

3.8369

In [179]:
df_trades

,symbol,id,orderId,side,price,qty,realizedPnl,marginAsset,quoteQty,commission,commissionAsset,time,positionSide,buyer,maker
53,APTUSDT,65514388,1253587098,BUY,3.83690,21.9,0,USDT,84.028110,0.03361124,USDT,1672761540276,BOTH,True,False


In [180]:
qty

21.9

In [174]:
# price_rounding = len(str(float([i["tickSize"] for i in ticker_info["filters"] if i["filterType"] == "PRICE_FILTER"][0])).split(".")[-1])
price_rounding = len(str(real_price_bought).split(".")[-1])
price_rounding

4

In [175]:
len(str(float([i["tickSize"] for i in ticker_info["filters"] if i["filterType"] == "PRICE_FILTER"][0])).split(".")[-1])

4

In [176]:
try:
    price_sell = round(real_price_bought*pct, price_rounding)
    real_price_bought, price_sell

    orderSell = bi.client.futures_create_order(
            symbol = symbol,
            type = "LIMIT", 
            timeInForce ="GTC",
            side = "SELL",
            price = price_sell, 
            quantity = qty,
        )
except Exception as e:
    print(e)